In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import pathlib
from util import runGridSearchClassifiers

## Loading data

In [3]:
directory_path = '../../04_-_Dev/videos'
features = 'emobase'

In [4]:
df_total = pd.read_pickle(directory_path + '/audio_' + features + '_data.p')

In [4]:
df_total.shape

(2573701, 60)

In [5]:
df_total.video_name.nunique()

30

In [6]:
df_total[df_total.isna().any(axis=1)]

,frameIndex,frameTime,pcm_intensity_sma,pcm_loudness_sma,mfcc_sma[1],mfcc_sma[2],mfcc_sma[3],mfcc_sma[4],mfcc_sma[5],mfcc_sma[6],...,pcm_zcr_sma_de,voiceProb_sma_de,F0_sma_de,F0env_sma_de,video_name,stress_global,type_candidat,sexe,stress,diapo


## Data processing

In [7]:
time_window = 5
df_total['frameTimeWindow'] = df_total.frameTime.apply(lambda x : np.floor(x / time_window) * time_window).astype(int)

In [33]:
from scipy.stats import kurtosis, skew

def percentil25(x): 
    return np.percentile(x, q=25)

def percentil75(x): 
    return np.percentile(x, q=75)

In [11]:
# 5 seconds windows
X = df_total.iloc[:,2:].groupby(['video_name','diapo','frameTimeWindow']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-18]

In [16]:
y = df_total.iloc[:,3:].groupby(['video_name','diapo','frameTimeWindow']).agg({'stress':'min'}).iloc[:,-1]

In [18]:
X.to_pickle(directory_path + '/audio_' + features + '_tw5_data_X.p')
y.to_pickle(directory_path + '/audio_' + features + '_tw5_data_y.p')

In [21]:
from scipy.stats import kurtosis, skew
X_audio = df_total.loc[df_total['diapo'].isin([8, 9, 10, 11, 17]),:].iloc[:,2:].groupby(['video_name','diapo','frameTimeWindow']).agg({'mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew}).iloc[:,:-18]
y_audio = df_total.loc[df_total['diapo'].isin([8, 9, 10, 11, 17]),:].iloc[:,2:].groupby(['video_name','diapo','frameTimeWindow']).agg({'stress':'mean'}).iloc[:,-1]

In [22]:
X_audio.to_pickle(directory_path + '/audio_' + features + '_tw5_data_X_audio_questions_only.p')
y_audio.to_pickle(directory_path + '/audio_' + features + '_tw5_data_y_audio_questions_only.p')

## Modèles

In [5]:
X = pd.read_pickle(directory_path + '/audio_' + features + '_tw5_data_X.p')
y = pd.read_pickle(directory_path + '/audio_' + features + '_tw5_data_y.p')

X_audio = pd.read_pickle(directory_path + '/audio_' + features + '_tw5_data_X_audio_questions_only.p')
y_audio = pd.read_pickle(directory_path + '/audio_' + features + '_tw5_data_y_audio_questions_only.p')

In [6]:
X = X.fillna(0)
X_audio = X_audio.fillna(0)

## Stress par diapos
### All diapos

In [127]:
diapo_selection = '_all'

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [RandomForestClassifier(random_state = 42, n_jobs=-1)]
parameters_list = [
                {'n_estimators': [100, 150, 200, 250, 300], 'max_depth':[10, 15, 20, 25,30], 'class_weight':[None,'balanced']}
                ]
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [17]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 50 candidates, totalling 1500 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 24.4min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 45.7min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 71.5min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 89.9min finished
Best estimator RandomForestClassifier(class_weight='balanced', max_depth=15, n_estimators=200,
                       n_jobs=-1, random_state=42)
Best results 0.4529982783484758
Best params {'class_weight': 'balanced', 'max_depth': 15, 'n_estimators': 200}
accuracy (mean, std) 0.4603521332304298 0.2114706580772435
f1 (mean, std) 0.4529982783484758 0.20974192525992463
balanced accuracy (mean, std) 0.43354841847414627 0.17316512582141624
precision (mean, std) 0.572314966759

In [22]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name','diapo','frameTimeWindow']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')

In [23]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predict)

array([[ 654, 1239,  171,    0],
       [ 814, 1749,   27,    0],
       [ 316,  307,   30,    0],
       [   7,   13,    0,    0]])

In [129]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','frameTimeWindow','ypredict']
df_ypredict['ypredict'] = df_ypredict['ypredict'].astype(int)


In [130]:
X

,"(ypredict, mean)","(ypredict, min)","(ypredict, max)","(ypredict, median)","(ypredict, std)","(ypredict, percentil25)","(ypredict, percentil75)","(ypredict, kurtosis)","(ypredict, skew)"
video_name,,,,,,,,,
Test_pour_AFPA,0.943662,0.0,1.0,1.0,0.232214,1.0,1.0,12.809701,-3.848337
Video_1,0.236111,0.0,1.0,0.0,0.427672,0.0,0.0,-0.455615,1.242733
WIN_20210323_19_17_40_Pro,0.472222,0.0,1.0,0.0,0.502731,0.0,1.0,-1.987616,0.111283
WIN_20210329_10_16_02_Pro,0.930556,0.0,1.0,1.0,0.255992,1.0,1.0,9.474627,-3.387422
WIN_20210330_13_10_29_Pro,0.958904,0.0,1.0,1.0,0.199886,1.0,1.0,19.376190,-4.623439
WIN_20210331_21_22_52_Pro,0.901408,0.0,1.0,1.0,0.300235,1.0,1.0,5.252232,-2.692997
WIN_20210402_14_27_50_Pro,1.847222,1.0,2.0,2.0,0.362298,2.0,2.0,1.725782,-1.930229
WIN_20210402_19_04_53_Pro,0.680556,0.0,1.0,1.0,0.469533,0.0,1.0,-1.400177,-0.774482
WIN_20210403_18_49_15_Pro,0.722222,0.0,1.0,1.0,0.451046,0.0,1.0,-1.015385,-0.992278


#### En utilisant la proportion des prédictions 0, 1 et 2

In [131]:
X = df_ypredict.pivot_table(values='frameTimeWindow', columns='ypredict', index=['video_name','diapo'], aggfunc='count', fill_value=0)
X_sum = X.sum(axis=1).values.copy()
for col_number in range(len(X.columns)):
    X.iloc[:,col_number] = X.iloc[:,col_number] / X_sum
X
 

ypredict                                0         1     2
video_name                diapo                          
Test_pour_AFPA            1      0.176471  0.823529  0.00
                          8      0.200000  0.800000  0.00
                          9      0.000000  0.950000  0.05
                          10     0.000000  1.000000  0.00
                          11     0.050000  0.950000  0.00
...                                   ...       ...   ...
WIN_20210417_14_53_12_Pro 10     0.428571  0.571429  0.00
                          11     0.600000  0.400000  0.00
                          12     0.151515  0.848485  0.00
                          17     0.571429  0.428571  0.00
                          18     0.304348  0.695652  0.00

[240 rows x 3 columns]

In [132]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(df_annotations_stress.columns[-1],axis=1).set_index('video_name').stack()
df_annotations_stress = pd.DataFrame(df_annotations_stress).reset_index()
df_annotations_stress.columns = ['video_name','diapo','stress']
#df_annotations_stress = df_annotations_stress.set_index(['video_name','diapo'])
df_annotations_stress['diapo'] = df_annotations_stress['diapo'].astype(int)
df_annotations_stress

,video_name,diapo,stress
0,Test_pour_AFPA,1,1.0
1,Test_pour_AFPA,8,1.0
2,Test_pour_AFPA,9,0.0
3,Test_pour_AFPA,10,0.0
4,Test_pour_AFPA,11,0.0
...,...,...,...
235,WIN_20210417_14_53_12_Pro,10,0.0
236,WIN_20210417_14_53_12_Pro,11,0.0
237,WIN_20210417_14_53_12_Pro,12,0.0
238,WIN_20210417_14_53_12_Pro,17,1.0


In [133]:
Xy = X.merge(df_annotations_stress, how='inner', on=['video_name','diapo'])
X = Xy.iloc[:,:-1].set_index(['video_name','diapo'])
y = Xy.iloc[:,-1]

In [134]:
X

0         1     2
video_name                diapo                          
Test_pour_AFPA            1      0.176471  0.823529  0.00
                          8      0.200000  0.800000  0.00
                          9      0.000000  0.950000  0.05
                          10     0.000000  1.000000  0.00
                          11     0.050000  0.950000  0.00
...                                   ...       ...   ...
WIN_20210417_14_53_12_Pro 10     0.428571  0.571429  0.00
                          11     0.600000  0.400000  0.00
                          12     0.151515  0.848485  0.00
                          17     0.571429  0.428571  0.00
                          18     0.304348  0.695652  0.00

[240 rows x 3 columns]

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [32]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.342090687090687
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.4666666666666667 0.24986107250941583
f1 (mean, std) 0.342090687090687 0.26396459108072284
balanced accuracy (mean, std) 0.45595238095238094 0.10688460940608407
precision (mean, std) 0.2864583333333333 0.25224684779442186
recall (mean, std) 0.4666666666666667 0.24986107250941583

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 2160 tasks      | elapsed:    8.

In [33]:
best_result

{'best_estimator': KNeighborsClassifier(),
 'best_score': 0.5049107142857142,
 'best_params': {'n_neighbors': 5, 'p': 2, 'weights': 'uniform'},
 'mean_test_f1_score': 0.5049107142857142,
 'std_test_f1_score': 0.22926693101143997,
 'mean_test_accuracy_score': 0.5041666666666667,
 'std_test_accuracy_score': 0.20021689627889938,
 'mean_test_balanced_accuracy_score': 0.5370899470899472,
 'std_test_balanced_accuracy_score': 0.20378238534037207,
 'mean_test_precision': 0.6314136904761904,
 'std_test_precision': 0.28049332460208276,
 'mean_test_recall': 0.5041666666666667,
 'std_test_recall': 0.20021689627889938}

#### En aggrégeant les prédicitions des diapos

In [135]:
# Autre méthode
X = df_ypredict.groupby(['video_name','diapo']).agg({'ypredict':['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})

In [136]:
X

ypredict                           \
                                     mean min max median       std   
video_name                diapo                                      
Test_pour_AFPA            1      0.823529   0   1    1.0  0.386953   
                          8      0.800000   0   1    1.0  0.421637   
                          9      1.050000   1   2    1.0  0.223607   
                          10     1.000000   1   1    1.0  0.000000   
                          11     0.950000   0   1    1.0  0.223607   
...                                   ...  ..  ..    ...       ...   
WIN_20210417_14_53_12_Pro 10     0.571429   0   1    1.0  0.513553   
                          11     0.400000   0   1    0.0  0.502625   
                          12     0.848485   0   1    1.0  0.364110   
                          17     0.428571   0   1    0.0  0.534522   
                          18     0.695652   0   1    1.0  0.470472   

                                                                              
                                percentil25 percentil75   kurtosis      skew  
video_name                diapo                                               
Test_pour_AFPA            1             1.0         1.0   0.880952 -1.697337  
                          8             1.0         1.0   0.250000 -1.500000  
                          9             1.0         1.0  15.052632  4.129483  
                          10            1.0         1.0  -3.000000  0.000000  
                          11            1.0         1.0  15.052632 -4.129483  
...                                     ...         ...        ...       ...  
WIN_20210417_14_53_12_Pro 10            0.0         1.0  -1.916667 -0.288675  
                          11            0.0         1.0  -1.833333  0.408248  
                          12            1.0         1.0   1.778571 -1.943855  
                          17            0.0         1.0  -1.916667  0.288675  
                          18            0.0         1.0  -1.276786 -0.850420  

[240 rows x 9 columns]

In [137]:
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [138]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)
best_result

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    9.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=4, multi_class='multinomial', random_state=42)
Best results 0.4583969271469271
Best params {'C': 4, 'class_weight': None}
accuracy (mean, std) 0.4708333333333333 0.2033145483125975
f1 (mean, std) 0.4583969271469271 0.22590056284071497
balanced accuracy (mean, std) 0.49503968253968256 0.18076355105014674
precision (mean, std) 0.568531746031746 0.3019076277002293
recall (mean, std) 0.4708333333333333 0.2033145483125975

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.6s
[Paral

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=4)),
                 ('knn',
                  KNeighborsClassifier(n_neighbors=9, weights='distance'))]),
 'best_score': 0.5105066923816923,
 'best_params': {'knn__n_neighbors': 9,
  'knn__p': 2,
  'knn__weights': 'distance',
  'pca__n_components': 4},
 'mean_test_f1_score': 0.5105066923816923,
 'std_test_f1_score': 0.20498075270089106,
 'mean_test_accuracy_score': 0.5,
 'std_test_accuracy_score': 0.18257418583505536,
 'mean_test_balanced_accuracy_score': 0.49427248677248675,
 'std_test_balanced_accuracy_score': 0.20013975032987208,
 'mean_test_precision': 0.6111855158730158,
 'std_test_precision': 0.2507258574113345,
 'mean_test_recall': 0.5,
 'std_test_recall': 0.18257418583505536}

In [139]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=4)),
                 ('knn',
                  KNeighborsClassifier(n_neighbors=9, weights='distance'))]),
 'best_score': 0.5105066923816923,
 'best_params': {'knn__n_neighbors': 9,
  'knn__p': 2,
  'knn__weights': 'distance',
  'pca__n_components': 4},
 'mean_test_f1_score': 0.5105066923816923,
 'std_test_f1_score': 0.20498075270089106,
 'mean_test_accuracy_score': 0.5,
 'std_test_accuracy_score': 0.18257418583505536,
 'mean_test_balanced_accuracy_score': 0.49427248677248675,
 'std_test_balanced_accuracy_score': 0.20013975032987208,
 'mean_test_precision': 0.6111855158730158,
 'std_test_precision': 0.2507258574113345,
 'mean_test_recall': 0.5,
 'std_test_recall': 0.18257418583505536}

In [140]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predict)

array([[47, 50,  4,  0],
       [42, 63,  7,  1],
       [ 6, 19,  0,  0],
       [ 0,  1,  0,  0]])

In [141]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name','diapo']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_tw5_diapo' + diapo_selection + '.csv')

In [142]:
df_ypredict

,"(video_name, )","(diapo, )",ypredict
0,Test_pour_AFPA,1,1.0
1,Test_pour_AFPA,8,1.0
2,Test_pour_AFPA,9,1.0
3,Test_pour_AFPA,10,0.0
4,Test_pour_AFPA,11,1.0
...,...,...,...
235,WIN_20210417_14_53_12_Pro,10,1.0
236,WIN_20210417_14_53_12_Pro,11,0.0
237,WIN_20210417_14_53_12_Pro,12,1.0
238,WIN_20210417_14_53_12_Pro,17,0.0


## Stress global

### En utilisant le stress prédit des diapos

In [11]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5_diapo' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','ypredict']
ypredict_stress_diapo = df_ypredict.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')


In [12]:
ypredict_stress_diapo

diapo,1,8,9,10,11,12,17,18
video_name,,,,,,,,
Test_pour_AFPA,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
Video_1,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
WIN_20210323_19_17_40_Pro,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
WIN_20210329_10_16_02_Pro,1.0,0.0,1.0,0.0,1.0,2.0,1.0,0.0
WIN_20210330_13_10_29_Pro,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0
WIN_20210331_21_22_52_Pro,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
WIN_20210402_14_27_50_Pro,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
WIN_20210402_19_04_53_Pro,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0
WIN_20210403_18_49_15_Pro,0.0,0.0,0.0,0.0,3.0,1.0,1.0,1.0


In [13]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.set_index(['video_name'])
df_annotations_stress

,1,8,9,10,11,12,17,18,stress_global
video_name,,,,,,,,,
Test_pour_AFPA,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
Video_1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
WIN_20210323_19_17_40_Pro,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
WIN_20210329_10_16_02_Pro,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
WIN_20210330_13_10_29_Pro,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
WIN_20210331_21_22_52_Pro,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
WIN_20210402_14_27_50_Pro,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
WIN_20210402_19_04_53_Pro,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0
WIN_20210403_18_49_15_Pro,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0


In [16]:
Xy = ypredict_stress_diapo.merge(df_annotations_stress.iloc[:,-1],how='inner',on='video_name')
X = Xy.iloc[:,:-1]
y = Xy.iloc[:,-1]

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [20]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.4
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.4 0.4898979485566357
f1 (mean, std) 0.4 0.4898979485566357
balanced accuracy (mean, std) 0.4 0.4898979485566357
precision (mean, std) 0.4 0.4898979485566357
recall (mean, std) 0.4 0.4898979485566357

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:    9.1s finished
[Parallel(n_jobs=-

In [22]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('knn',
                  KNeighborsClassifier(n_neighbors=7, p=1, weights='distance'))]),
 'best_score': 0.5,
 'best_params': {'knn__n_neighbors': 7,
  'knn__p': 1,
  'knn__weights': 'distance',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.5,
 'std_test_f1_score': 0.5,
 'mean_test_accuracy_score': 0.5,
 'std_test_accuracy_score': 0.5,
 'mean_test_balanced_accuracy_score': 0.5,
 'std_test_balanced_accuracy_score': 0.5,
 'mean_test_precision': 0.5,
 'std_test_precision': 0.5,
 'mean_test_recall': 0.5,
 'std_test_recall': 0.5}

In [23]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_tw5_global' + diapo_selection + '.csv')

#### Autre méthode

### En utilisant le stress prédit des time windows 5s

In [60]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','frameTimeWindow','ypredict']
df_ypredict['ypredict'] = df_ypredict['ypredict'].astype(int)
df_ypredict = df_ypredict.pivot_table(values='frameTimeWindow', columns='ypredict', index='video_name', aggfunc='count', fill_value=0)
df_ypredict_sum = df_ypredict.sum(axis=1).values.copy()
for col_number in range(len(df_ypredict.columns)):
    df_ypredict.iloc[:,col_number] = df_ypredict.iloc[:,col_number] / df_ypredict_sum
#df_ypredict = df_ypredict.reset_index()
df_ypredict


ypredict,0,1,2
video_name,,,
Test_pour_AFPA,0.185185,0.798942,0.015873
Video_1,0.357616,0.642384,0.000000
WIN_20210323_19_17_40_Pro,0.303571,0.690476,0.005952
WIN_20210329_10_16_02_Pro,0.190476,0.809524,0.000000
WIN_20210330_13_10_29_Pro,0.271523,0.728477,0.000000
WIN_20210331_21_22_52_Pro,0.329412,0.670588,0.000000
WIN_20210402_14_27_50_Pro,0.005376,0.994624,0.000000
WIN_20210402_19_04_53_Pro,0.897143,0.080000,0.022857
WIN_20210403_18_49_15_Pro,0.287293,0.712707,0.000000


In [61]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress[['video_name','stress_global']]

In [70]:
Xy = df_ypredict.merge(df_annotations_stress, on='video_name')
X = Xy.iloc[:,:-1].set_index('video_name')
y = Xy.iloc[:,-1]

In [71]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [72]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.9s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.4
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.4 0.4898979485566357
f1 (mean, std) 0.4 0.4898979485566357
balanced accuracy (mean, std) 0.4 0.4898979485566357
precision (mean, std) 0.4 0.4898979485566357
recall (mean, std) 0.4 0.4898979485566357

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 2160 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:    7.8s finished
[Parallel(n_jobs=-

In [73]:
best_result

{'best_estimator': KNeighborsClassifier(n_neighbors=9),
 'best_score': 0.5,
 'best_params': {'n_neighbors': 9, 'p': 2, 'weights': 'uniform'},
 'mean_test_f1_score': 0.5,
 'std_test_f1_score': 0.5,
 'mean_test_accuracy_score': 0.5,
 'std_test_accuracy_score': 0.5,
 'mean_test_balanced_accuracy_score': 0.5,
 'std_test_balanced_accuracy_score': 0.5,
 'mean_test_precision': 0.5,
 'std_test_precision': 0.5,
 'mean_test_recall': 0.5,
 'std_test_recall': 0.5}

#### Autre méthode

In [75]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','frameTimeWindow','ypredict']
df_ypredict = df_ypredict.groupby(['video_name']).agg({'ypredict': ['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})
df_ypredict


ypredict                             \
                                     mean  min  max median       std   
video_name                                                             
Test_pour_AFPA                   0.830688  0.0  2.0    1.0  0.416303   
Video_1                          0.642384  0.0  1.0    1.0  0.480893   
WIN_20210323_19_17_40_Pro        0.702381  0.0  2.0    1.0  0.471455   
WIN_20210329_10_16_02_Pro        0.809524  0.0  1.0    1.0  0.394019   
WIN_20210330_13_10_29_Pro        0.728477  0.0  1.0    1.0  0.446225   
WIN_20210331_21_22_52_Pro        0.670588  0.0  1.0    1.0  0.471388   
WIN_20210402_14_27_50_Pro        0.994624  0.0  1.0    1.0  0.073324   
WIN_20210402_19_04_53_Pro        0.125714  0.0  2.0    0.0  0.395625   
WIN_20210403_18_49_15_Pro        0.712707  0.0  1.0    1.0  0.453755   
WIN_20210404_10_58_27_Pro        0.265000  0.0  2.0    0.0  0.588649   
WIN_20210404_21_41_12_Pro        0.650862  0.0  1.0    1.0  0.477728   
WIN_20210405_15_09_16_Pro        0.606452  0.0  2.0    1.0  0.503195   
WIN_20210406_15_06_15_Pro        0.863415  0.0  1.0    1.0  0.344250   
WIN_20210406_18_35_52_Pro        0.518987  0.0  1.0    1.0  0.501228   
WIN_20210406_18_49_10_Pro        0.783626  0.0  1.0    1.0  0.412982   
WIN_20210406_21_05_52_Pro        0.589189  0.0  1.0    1.0  0.493316   
WIN_20210407_09_04_05_Pro        0.565445  0.0  2.0    1.0  0.507481   
WIN_20210407_14_54_56_Pro_edit2  0.616352  0.0  1.0    1.0  0.487810   
WIN_20210408_11_48_58_Pro        1.293333  0.0  2.0    1.0  0.537188   
WIN_20210408_14_00_44_Pro        0.814607  0.0  1.0    1.0  0.389713   
WIN_20210408_14_02_19_Pro        0.582353  0.0  2.0    1.0  0.506450   
WIN_20210408_14_11_32_Pro        1.000000  1.0  1.0    1.0  0.000000   
WIN_20210408_15_20_51_Pro        1.500000  0.0  2.0    2.0  0.705363   
WIN_20210408_16_04_32_Pro        0.864706  0.0  1.0    1.0  0.343048   
WIN_20210409_10_26_11_Pro        0.497110  0.0  1.0    0.0  0.501443   
WIN_20210413_15_38_01_Pro        0.745665  0.0  1.0    1.0  0.436751   
WIN_20210414_06_24_52_Pro        0.005181  0.0  1.0    0.0  0.071982   
WIN_20210415_15_41_24_Pro        0.488479  0.0  1.0    0.0  0.501023   
WIN_20210416_08_06_54_Pro        1.000000  1.0  1.0    1.0  0.000000   
WIN_20210417_14_53_12_Pro        0.595420  0.0  1.0    1.0  0.492695   

                                                                                
                                percentil25 percentil75    kurtosis       skew  
video_name                                                                      
Test_pour_AFPA                          1.0         1.0    0.987649  -1.074290  
Video_1                                 0.0         1.0   -1.647003  -0.594136  
WIN_20210323_19_17_40_Pro               0.0         1.0   -1.029843  -0.712361  
WIN_20210329_10_16_02_Pro               1.0         1.0    0.485294  -1.576482  
WIN_20210330_13_10_29_Pro               0.0         1.0   -0.944346  -1.027450  
WIN_20210331_21_22_52_Pro               0.0         1.0   -1.473058  -0.725908  
WIN_20210402_14_27_50_Pro               1.0         1.0  181.005405 -13.527949  
WIN_20210402_19_04_53_Pro               0.0         0.0   10.586080   3.293186  
WIN_20210403_18_49_15_Pro               0.0         1.0   -1.116130  -0.940144  
WIN_20210404_10_58_27_Pro               0.0         0.0    3.033203   2.085987  
WIN_20210404_21_41_12_Pro               0.0         1.0   -1.599379  -0.632947  
WIN_20210405_15_09_16_Pro               0.0         1.0   -1.535670  -0.281970  
WIN_20210406_15_06_15_Pro               1.0         1.0    2.479621  -2.116511  
WIN_20210406_18_35_52_Pro               0.0         1.0   -1.994223  -0.076004  
WIN_20210406_18_49_10_Pro               1.0         1.0   -0.102259  -1.377585  
WIN_20210406_21_05_52_Pro               0.0         1.0   -1.868542  -0.362572  
WIN_20210407_09_04_05_Pro               0.0         1.0   -1.675625  -0.142755  
WIN_20210407_14_54_56_Pro_edit2     

In [76]:
Xy = df_ypredict.merge(df_annotations_stress, on='video_name')
X = Xy.iloc[:,:-1].set_index('video_name')
y = Xy.iloc[:,-1]

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [22]:
parameters = {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 10, 20], 'class_weight' : [None, 'balanced']}
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings("ignore")

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)
X_no_name = X
y_no_name = y

model = LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42)

clf = GridSearchCV(estimator=model, 
                    param_grid=parameters, 
                    scoring={'accuracy_score' : 'accuracy', 'f1_score' : 'f1_weighted' }, 
                    refit='accuracy_score',
                    cv=5, verbose=1)
clf.fit(X_no_name, y_no_name)
print('Best results', clf.best_score_)
print('Best params', clf.best_params_)
print('accuracy (mean, std)', clf.cv_results_['mean_test_accuracy_score'][clf.best_index_], clf.cv_results_['std_test_accuracy_score'][clf.best_index_])
print('f1 (mean, std)', clf.cv_results_['mean_test_f1_score'][clf.best_index_], clf.cv_results_['std_test_f1_score'][clf.best_index_])

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Best results 0.4999999999999999
Best params {'C': 10, 'class_weight': None}
accuracy (mean, std) 0.4999999999999999 0.14907119849998596
f1 (mean, std) 0.4666666666666666 0.14622830112321866
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.8s finished


In [80]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 560 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 585 out of 600 | elapsed:    6.7s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    6.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, class_weight='balanced', multi_class='multinomial',
                   random_state=42)
Best results 0.4
Best params {'C': 0.01, 'class_weight': 'balanced'}
accuracy (mean, std) 0.4 0.4898979485566357
f1 (mean, std) 0.4 0.4898979485566357
balanced accuracy (mean, std) 0.4 0.4898979485566357
precision (mean, std) 0.4 0.4898979485566357
recall (mean, std) 0.4 0.4898979485566357

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: 

In [82]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=3)),
                 ('logistic',
                  LogisticRegression(C=0.1, class_weight='balanced',
                                     multi_class='multinomial'))]),
 'best_score': 0.4666666666666667,
 'best_params': {'logistic__C': 0.1,
  'logistic__class_weight': 'balanced',
  'pca__n_components': 3},
 'mean_test_f1_score': 0.4666666666666667,
 'std_test_f1_score': 0.49888765156985887,
 'mean_test_accuracy_score': 0.4666666666666667,
 'std_test_accuracy_score': 0.49888765156985887,
 'mean_test_balanced_accuracy_score': 0.4666666666666667,
 'std_test_balanced_accuracy_score': 0.49888765156985887,
 'mean_test_precision': 0.4666666666666667,
 'std_test_precision': 0.49888765156985887,
 'mean_test_recall': 0.4666666666666667,
 'std_test_recall': 0.49888765156985887}

In [26]:
df_ypredict_stress_global = pd.concat([ypredict_stress_diapo.reset_index(), pd.DataFrame(ypredict_stress_global,columns=['predicted_stress_global'])], axis=1) 
df_ypredict_stress_global = df_ypredict_stress_global.set_index('video_name').sort_index()
df_ypredict_stress_global = df_ypredict_stress_global.iloc[:,-1]

In [27]:
df_ypredict_stress_global

video_name
Test_pour_AFPA                     1.0
Video_1                            0.0
WIN_20210323_19_17_40_Pro          0.0
WIN_20210329_10_16_02_Pro          1.0
WIN_20210330_13_10_29_Pro          0.0
WIN_20210331_21_22_52_Pro          0.0
WIN_20210402_14_27_50_Pro          1.0
WIN_20210402_19_04_53_Pro          2.0
WIN_20210403_18_49_15_Pro          0.0
WIN_20210404_10_58_27_Pro          0.0
WIN_20210404_21_41_12_Pro          0.0
WIN_20210405_15_09_16_Pro          0.0
WIN_20210406_15_06_15_Pro          1.0
WIN_20210406_18_35_52_Pro          0.0
WIN_20210406_18_49_10_Pro          1.0
WIN_20210406_21_05_52_Pro          0.0
WIN_20210407_09_04_05_Pro          0.0
WIN_20210407_14_54_56_Pro_edit2    0.0
WIN_20210408_11_48_58_Pro          0.0
WIN_20210408_14_00_44_Pro          1.0
WIN_20210408_14_02_19_Pro          0.0
WIN_20210408_14_11_32_Pro          1.0
WIN_20210408_15_20_51_Pro          0.0
WIN_20210408_16_04_32_Pro          1.0
WIN_20210409_10_26_11_Pro          0.0
WIN_20210413_1

### Audios diapos only

In [75]:
diapo_selection = '_audio_only'
diapo_audio_list = [8,9,10,11,17]
X = X_audio
y = y_audio

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut

models_list = [RandomForestClassifier(random_state = 42, n_jobs=-1)]
parameters_list = [
                {'n_estimators': [100, 150, 200], 'max_depth':[10, 15, 20, 25], 'class_weight':[None,'balanced']}
                ]
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [12]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 24 candidates, totalling 720 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed: 12.5min finished
Best estimator RandomForestClassifier(max_depth=25, n_jobs=-1, random_state=42)
Best results 0.4895567766570025
Best params {'class_weight': None, 'max_depth': 25, 'n_estimators': 100}
accuracy (mean, std) 0.4627812792533997 0.2374004254394037
f1 (mean, std) 0.4895567766570025 0.2643965616504656
balanced accuracy (mean, std) 0.47060091995098524 0.1975398259321073
precision (mean, std) 0.6905439806468997 0.29522573381181333
recall (mean, std) 0.4627812792533997 0.2374004254394037



In [13]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name','diapo','frameTimeWindow']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')

In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predict)

array([[463, 455,  20,   0],
       [396, 528,  51,   0],
       [117, 105,  12,   0],
       [  5,  15,   0,   0]])

In [16]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','frameTimeWindow','ypredict']
df_ypredict['ypredict'] = df_ypredict['ypredict'].astype(int)


In [17]:
X

pcm_intensity_sma  \
                                                      percentil75   
video_name                diapo frameTimeWindow                     
Test_pour_AFPA            8     170                  9.496449e-11   
                                175                  1.355299e-10   
                                180                  2.465382e-09   
                                185                  1.685430e-06   
                                190                  6.110832e-07   
...                                                           ...   
WIN_20210417_14_53_12_Pro 17    490                  6.222839e-07   
                                495                  6.578761e-07   
                                500                  4.799780e-08   
                                505                  0.000000e+00   
                                510                  0.000000e+00   

                                                                          \
                                                       median       skew   
video_name                diapo frameTimeWindow                            
Test_pour_AFPA            8     170              5.266774e-11   1.275324   
                                175              7.317847e-11  11.889553   
                                180              8.678470e-11   6.124526   
                                185              1.811864e-07   3.950305   
                                190              1.293229e-07   2.687117   
...                                                       ...        ...   
WIN_20210417_14_53_12_Pro 17    490              1.734076e-07   3.620979   
                                495              2.307396e-07   3.313799   
                                500              5.425397e-13   2.632982   
                                505              0.000000e+00  11.791190   
                                510              0.000000e+00   0.000000   

                                                                             \
                                                          std   percentil25   
video_name                diapo frameTimeWindow                               
Test_pour_AFPA            8     170              4.987511e-11  3.240534e-11   
                                175              1.017973e-07  3.820685e-11   
                                180              3.866010e-07  4.097161e-11   
                                185              2.889615e-06  2.326003e-08   
                                190              1.158892e-06  2.341649e-08   
...                                                       ...           ...   
WIN_20210417_14_53_12_Pro 17    490              1.145962e-06  3.047811e-08   
                                495              9.449784e-07  5.144652e-08   
                                500              4.625046e-07  0.000000e+00   
                                505              8.328170e-12  0.000000e+00   
                                510              0.000000e+00  0.000000e+00   

                                                                           \
                                                         mean    kurtosis   
video_name                diapo frameTimeWindow                             
Test_pour_AFPA            8     170              6.900340e-11    1.683537   
                                175              1.127223e-08  145.594452   
                                180              1.002494e-07   43.205763   
                                185              1.454266e-06   21.895735   
                                190              6.382472e-07    7.759795   
...                                                       ...         ...   
WIN_20210417_14_53_12_Pro 17    490              6.108926e-07   15.845811   
                                495              5.760812e-07   13.476700   
                                500              1.976648e-07 

#### En utilisant la proportion des prédictions 0, 1 et 2

In [18]:
X = df_ypredict.pivot_table(values='frameTimeWindow', columns='ypredict', index=['video_name','diapo'], aggfunc='count', fill_value=0)
X_sum = X.sum(axis=1).values.copy()
for col_number in range(len(X.columns)):
    X.iloc[:,col_number] = X.iloc[:,col_number] / X_sum
X
 

ypredict                                0         1    2
video_name                diapo                         
Test_pour_AFPA            8      0.000000  1.000000  0.0
                          9      0.150000  0.850000  0.0
                          10     0.066667  0.933333  0.0
                          11     0.000000  1.000000  0.0
                          17     0.000000  1.000000  0.0
...                                   ...       ...  ...
WIN_20210417_14_53_12_Pro 8      0.909091  0.090909  0.0
                          9      0.952381  0.047619  0.0
                          10     0.714286  0.285714  0.0
                          11     0.700000  0.300000  0.0
                          17     0.714286  0.285714  0.0

[150 rows x 3 columns]

In [23]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.drop(df_annotations_stress.columns[-1],axis=1).set_index('video_name').stack()
df_annotations_stress = pd.DataFrame(df_annotations_stress).reset_index()
df_annotations_stress.columns = ['video_name','diapo','stress']
#df_annotations_stress = df_annotations_stress.set_index(['video_name','diapo'])
df_annotations_stress['diapo'] = df_annotations_stress['diapo'].astype(int)
df_annotations_stress = df_annotations_stress[df_annotations_stress.diapo.isin(diapo_audio_list)]
df_annotations_stress

,video_name,diapo,stress
1,Test_pour_AFPA,8,1.0
2,Test_pour_AFPA,9,0.0
3,Test_pour_AFPA,10,0.0
4,Test_pour_AFPA,11,0.0
6,Test_pour_AFPA,17,0.0
...,...,...,...
233,WIN_20210417_14_53_12_Pro,8,0.0
234,WIN_20210417_14_53_12_Pro,9,0.0
235,WIN_20210417_14_53_12_Pro,10,0.0
236,WIN_20210417_14_53_12_Pro,11,0.0


In [24]:
Xy = X.merge(df_annotations_stress, how='inner', on=['video_name','diapo'])
X = Xy.iloc[:,:-1].set_index(['video_name','diapo'])
y = Xy.iloc[:,-1]

In [28]:
X

(pcm_intensity_sma, percentil75)  \
video_name                diapo                                     
Test_pour_AFPA            8                          9.496449e-11   
                          8                          1.355299e-10   
                          8                          2.465382e-09   
                          8                          1.685430e-06   
                          8                          6.110832e-07   
...                                                           ...   
WIN_20210417_14_53_12_Pro 17                         6.222839e-07   
                          17                         6.578761e-07   
                          17                         4.799780e-08   
                          17                         0.000000e+00   
                          17                         0.000000e+00   

                                 (pcm_intensity_sma, median)  \
video_name                diapo                                
Test_pour_AFPA            8                     5.266774e-11   
                          8                     7.317847e-11   
                          8                     8.678470e-11   
                          8                     1.811864e-07   
                          8                     1.293229e-07   
...                                                      ...   
WIN_20210417_14_53_12_Pro 17                    1.734076e-07   
                          17                    2.307396e-07   
                          17                    5.425397e-13   
                          17                    0.000000e+00   
                          17                    0.000000e+00   

                                 (pcm_intensity_sma, skew)  \
video_name                diapo                              
Test_pour_AFPA            8                       1.275324   
                          8                      11.889553   
                          8                       6.124526   
                          8                       3.950305   
                          8                       2.687117   
...                                                    ...   
WIN_20210417_14_53_12_Pro 17                      3.620979   
                          17                      3.313799   
                          17                      2.632982   
                          17                     11.791190   
                          17                      0.000000   

                                 (pcm_intensity_sma, std)  \
video_name                diapo                             
Test_pour_AFPA            8                  4.987511e-11   
                          8                  1.017973e-07   
                          8                  3.866010e-07   
                          8                  2.889615e-06   
                          8                  1.158892e-06   
...                                                   ...   
WIN_20210417_14_53_12_Pro 17                 1.145962e-06   
                          17                 9.449784e-07   
                          17                 4.625046e-07   
                          17                 8.328170e-12   
                          17                 0.000000e+00   

                                 (pcm_intensity_sma, percentil25)  \
video_name                diapo                                     
Test_pour_AFPA            8                          3.240534e-11   
                          8                          3.820685e-11   
                          8                          4.097161e-11   
                          8                          2.326003e-08   
                          8                          2.341649e-08   
...                                                           ...   
WIN_20210417_14_53_12_Pro 17                         3.047811e-08   
                          17                         5.144652e-08   
           

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
pipe = Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42))])

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [30]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:  1.2min finished
Best estimator LogisticRegression(C=0.1, multi_class='multinomial', random_state=42)
Best results 0.5222474920391509
Best params {'C': 0.1, 'class_weight': None}
accuracy (mean, std) 0.4833192202428166 0.22080345139876076
f1 (mean, std) 0.5222474920391509 0.24059668385562108
balanced accuracy (mean, std) 0.4666813458197785 0.18519366298874185
precision (mean, std) 0.6710389436987686 0.29155896747966625
recall (mean, std) 0.4833192202428166 0.22080345139876076

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.

In [31]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=4)),
                 ('logistic',
                  LogisticRegression(C=0.5, multi_class='multinomial'))]),
 'best_score': 0.5248991779918025,
 'best_params': {'logistic__C': 0.5,
  'logistic__class_weight': None,
  'pca__n_components': 4},
 'mean_test_f1_score': 0.5248991779918025,
 'std_test_f1_score': 0.2121989956437543,
 'mean_test_accuracy_score': 0.47376501525642944,
 'std_test_accuracy_score': 0.20059257286883184,
 'mean_test_balanced_accuracy_score': 0.46050991470322106,
 'std_test_balanced_accuracy_score': 0.17026350671120616,
 'mean_test_precision': 0.6894235261711981,
 'std_test_precision': 0.2937294906716355,
 'mean_test_recall': 0.47376501525642944,
 'std_test_recall': 0.20059257286883184}

#### En aggrégeant les prédictions des diapos

In [34]:
# Autre méthode
X = df_ypredict.groupby(['video_name','diapo']).agg({'ypredict':['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})

In [40]:
Xy = X.merge(df_annotations_stress, how='inner', on=['video_name','diapo'])
X = Xy.iloc[:,:-1].set_index(['video_name','diapo'])
y = Xy.iloc[:,-1]

In [42]:
y

0      1.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
145    0.0
146    0.0
147    0.0
148    0.0
149    1.0
Name: stress, Length: 150, dtype: float64

In [43]:
groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [44]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)
best_result

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 560 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    8.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.05, multi_class='multinomial', random_state=42)
Best results 0.455021164021164
Best params {'C': 0.05, 'class_weight': None}
accuracy (mean, std) 0.4666666666666667 0.3112698007981
f1 (mean, std) 0.455021164021164 0.3250524089852367
balanced accuracy (mean, std) 0.4896296296296296 0.2990338810281261
precision (mean, std) 0.5756666666666668 0.3857816980087554
recall (mean, std) 0.4666666666666667 0.3112698007981

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.5s
[Parallel(

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('knn', KNeighborsClassifier(n_neighbors=4, p=1))]),
 'best_score': 0.5307142857142857,
 'best_params': {'knn__n_neighbors': 4,
  'knn__p': 1,
  'knn__weights': 'uniform',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.5307142857142857,
 'std_test_f1_score': 0.27125440357186104,
 'mean_test_accuracy_score': 0.5066666666666666,
 'std_test_accuracy_score': 0.2619584360585134,
 'mean_test_balanced_accuracy_score': 0.46898148148148144,
 'std_test_balanced_accuracy_score': 0.27220434874556015,
 'mean_test_precision': 0.6684444444444445,
 'std_test_precision': 0.327399103296735,
 'mean_test_recall': 0.5066666666666666,
 'std_test_recall': 0.2619584360585134}

In [45]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('knn', KNeighborsClassifier(n_neighbors=4, p=1))]),
 'best_score': 0.5307142857142857,
 'best_params': {'knn__n_neighbors': 4,
  'knn__p': 1,
  'knn__weights': 'uniform',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.5307142857142857,
 'std_test_f1_score': 0.27125440357186104,
 'mean_test_accuracy_score': 0.5066666666666666,
 'std_test_accuracy_score': 0.2619584360585134,
 'mean_test_balanced_accuracy_score': 0.46898148148148144,
 'std_test_balanced_accuracy_score': 0.27220434874556015,
 'mean_test_precision': 0.6684444444444445,
 'std_test_precision': 0.327399103296735,
 'mean_test_recall': 0.5066666666666666,
 'std_test_recall': 0.2619584360585134}

In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_predict)

array([[48, 18,  2,  0],
       [37, 29,  0,  0],
       [10,  5,  0,  0],
       [ 1,  0,  0,  0]])

In [49]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name','diapo']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_tw5_diapo' + diapo_selection + '.csv')

In [50]:
df_ypredict

,video_name,diapo,ypredict
0,Test_pour_AFPA,8,0.0
1,Test_pour_AFPA,9,0.0
2,Test_pour_AFPA,10,1.0
3,Test_pour_AFPA,11,0.0
4,Test_pour_AFPA,17,0.0
...,...,...,...
145,WIN_20210417_14_53_12_Pro,8,0.0
146,WIN_20210417_14_53_12_Pro,9,1.0
147,WIN_20210417_14_53_12_Pro,10,0.0
148,WIN_20210417_14_53_12_Pro,11,0.0


## Stress global

### En utilisant le stress prédit des diapos

In [51]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5_diapo' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','ypredict']
ypredict_stress_diapo = df_ypredict.pivot_table(values='ypredict', columns='diapo',index='video_name',aggfunc='mean')


In [52]:
ypredict_stress_diapo

diapo,8,9,10,11,17
video_name,,,,,
Test_pour_AFPA,0.0,0.0,1.0,0.0,0.0
Video_1,0.0,0.0,1.0,0.0,1.0
WIN_20210323_19_17_40_Pro,0.0,0.0,1.0,0.0,0.0
WIN_20210329_10_16_02_Pro,1.0,1.0,1.0,1.0,0.0
WIN_20210330_13_10_29_Pro,1.0,1.0,1.0,1.0,0.0
WIN_20210331_21_22_52_Pro,1.0,1.0,0.0,0.0,1.0
WIN_20210402_14_27_50_Pro,1.0,1.0,1.0,1.0,0.0
WIN_20210402_19_04_53_Pro,0.0,0.0,1.0,1.0,0.0
WIN_20210403_18_49_15_Pro,0.0,0.0,1.0,1.0,0.0


In [69]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress

,video_name,1,8,9,10,11,12,17,18,stress_global
0,Test_pour_AFPA,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1,Video_1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2,WIN_20210323_19_17_40_Pro,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,WIN_20210329_10_16_02_Pro,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
4,WIN_20210330_13_10_29_Pro,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,WIN_20210331_21_22_52_Pro,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
6,WIN_20210402_14_27_50_Pro,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0
7,WIN_20210402_19_04_53_Pro,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0
8,WIN_20210403_18_49_15_Pro,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0
9,WIN_20210404_10_58_27_Pro,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [92]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress.set_index(['video_name'])
diapo_audio_list.append('stress_global')
df_annotations_stress = df_annotations_stress[[str(diapo) for diapo in diapo_audio_list]]
diapo_audio_list.pop()
df_annotations_stress


,8,9,10,11,17,stress_global
video_name,,,,,,
Test_pour_AFPA,1.0,0.0,0.0,0.0,0.0,1.0
Video_1,0.0,0.0,0.0,1.0,1.0,0.0
WIN_20210323_19_17_40_Pro,0.0,1.0,1.0,1.0,1.0,1.0
WIN_20210329_10_16_02_Pro,0.0,1.0,1.0,0.0,0.0,1.0
WIN_20210330_13_10_29_Pro,0.0,0.0,0.0,1.0,0.0,0.0
WIN_20210331_21_22_52_Pro,0.0,1.0,1.0,1.0,0.0,1.0
WIN_20210402_14_27_50_Pro,1.0,1.0,2.0,1.0,1.0,1.0
WIN_20210402_19_04_53_Pro,1.0,1.0,1.0,1.0,1.0,2.0
WIN_20210403_18_49_15_Pro,1.0,1.0,1.0,1.0,2.0,2.0


In [93]:
Xy = ypredict_stress_diapo.merge(df_annotations_stress.iloc[:,-1],how='inner',on='video_name')
X = Xy.iloc[:,:-1]
y = Xy.iloc[:,-1]

In [94]:
Xy

,8,9,10,11,17,stress_global
video_name,,,,,,
Test_pour_AFPA,0.0,0.0,1.0,0.0,0.0,1.0
Video_1,0.0,0.0,1.0,0.0,1.0,0.0
WIN_20210323_19_17_40_Pro,0.0,0.0,1.0,0.0,0.0,1.0
WIN_20210329_10_16_02_Pro,1.0,1.0,1.0,1.0,0.0,1.0
WIN_20210330_13_10_29_Pro,1.0,1.0,1.0,1.0,0.0,0.0
WIN_20210331_21_22_52_Pro,1.0,1.0,0.0,0.0,1.0,1.0
WIN_20210402_14_27_50_Pro,1.0,1.0,1.0,1.0,0.0,1.0
WIN_20210402_19_04_53_Pro,0.0,0.0,1.0,1.0,0.0,2.0
WIN_20210403_18_49_15_Pro,0.0,0.0,1.0,1.0,0.0,2.0


In [95]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [96]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    4.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.4
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.4 0.4898979485566357
f1 (mean, std) 0.4 0.4898979485566357
balanced accuracy (mean, std) 0.4 0.4898979485566357
precision (mean, std) 0.4 0.4898979485566357
recall (mean, std) 0.4 0.4898979485566357

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1200 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 2400 out of 2400 | elapsed:    8.3s finished
[Parallel(n_jobs=-

In [99]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=4)),
                 ('knn',
                  KNeighborsClassifier(n_neighbors=20, p=1,
                                       weights='distance'))]),
 'best_score': 0.5333333333333333,
 'best_params': {'knn__n_neighbors': 20,
  'knn__p': 1,
  'knn__weights': 'distance',
  'pca__n_components': 4},
 'mean_test_f1_score': 0.5333333333333333,
 'std_test_f1_score': 0.49888765156985887,
 'mean_test_accuracy_score': 0.5333333333333333,
 'std_test_accuracy_score': 0.49888765156985887,
 'mean_test_balanced_accuracy_score': 0.5333333333333333,
 'std_test_balanced_accuracy_score': 0.49888765156985887,
 'mean_test_precision': 0.5333333333333333,
 'std_test_precision': 0.49888765156985887,
 'mean_test_recall': 0.5333333333333333,
 'std_test_recall': 0.49888765156985887}

In [100]:
# Saving predictions
df_ypredict = pd.concat([X.reset_index()[['video_name']],
                        pd.DataFrame(y_predict, columns=['ypredict'])],axis=1)
df_ypredict.to_csv('ypredict_' + features + '_tw5_global' + diapo_selection + '.csv')

#### Autre méthode

### En utilisant le stress prédit des time windows 5s

In [104]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','frameTimeWindow','ypredict']
df_ypredict['ypredict'] = df_ypredict['ypredict'].astype(int)
df_ypredict = df_ypredict.pivot_table(values='frameTimeWindow', columns='ypredict', index='video_name', aggfunc='count', fill_value=0)
df_ypredict_sum = df_ypredict.sum(axis=1).values.copy()
for col_number in range(len(df_ypredict.columns)):
    df_ypredict.iloc[:,col_number] = df_ypredict.iloc[:,col_number] / df_ypredict_sum
#df_ypredict = df_ypredict.reset_index()
df_ypredict


ypredict,0,1,2
video_name,,,
Test_pour_AFPA,0.056338,0.943662,0.000000
Video_1,0.763889,0.236111,0.000000
WIN_20210323_19_17_40_Pro,0.527778,0.472222,0.000000
WIN_20210329_10_16_02_Pro,0.069444,0.930556,0.000000
WIN_20210330_13_10_29_Pro,0.041096,0.958904,0.000000
WIN_20210331_21_22_52_Pro,0.098592,0.901408,0.000000
WIN_20210402_14_27_50_Pro,0.000000,0.152778,0.847222
WIN_20210402_19_04_53_Pro,0.319444,0.680556,0.000000
WIN_20210403_18_49_15_Pro,0.277778,0.722222,0.000000


In [105]:
df_annotations_stress = pd.read_csv('annotations.csv')
df_annotations_stress = df_annotations_stress[['video_name','stress_global']]

In [106]:
Xy = df_ypredict.merge(df_annotations_stress, on='video_name')
X = Xy.iloc[:,:-1].set_index('video_name')
y = Xy.iloc[:,-1]

In [110]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [111]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 585 out of 600 | elapsed:    2.4s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    2.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.01, multi_class='multinomial', random_state=42)
Best results 0.4
Best params {'C': 0.01, 'class_weight': None}
accuracy (mean, std) 0.4 0.4898979485566357
f1 (mean, std) 0.4 0.4898979485566357
balanced accuracy (mean, std) 0.4 0.4898979485566357
precision (mean, std) 0.4 0.4898979485566357
recall (mean, std) 0.4 0.4898979485566357

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 2160 tasks      | elapsed:    6.5s
[Parallel(

In [112]:
best_result

{'best_estimator': Pipeline(steps=[('pca', PCA(n_components=1)),
                 ('logistic',
                  LogisticRegression(C=0.01, class_weight='balanced',
                                     multi_class='multinomial'))]),
 'best_score': 0.5,
 'best_params': {'logistic__C': 0.01,
  'logistic__class_weight': 'balanced',
  'pca__n_components': 1},
 'mean_test_f1_score': 0.5,
 'std_test_f1_score': 0.5,
 'mean_test_accuracy_score': 0.5,
 'std_test_accuracy_score': 0.5,
 'mean_test_balanced_accuracy_score': 0.5,
 'std_test_balanced_accuracy_score': 0.5,
 'mean_test_precision': 0.5,
 'std_test_precision': 0.5,
 'mean_test_recall': 0.5,
 'std_test_recall': 0.5}

#### Autre méthode

In [119]:
df_ypredict = pd.read_csv('ypredict_' + features + '_tw5' + diapo_selection + '.csv')
df_ypredict = df_ypredict.iloc[:,1:]
df_ypredict.columns = ['video_name','diapo','frameTimeWindow','ypredict']
df_ypredict = df_ypredict.groupby(['video_name']).agg({'ypredict': ['mean','min','max', 'median', 'std', percentil25, percentil75, kurtosis, skew]})
df_ypredict


ypredict                             \
                                     mean  min  max median       std   
video_name                                                             
Test_pour_AFPA                   0.943662  0.0  1.0    1.0  0.232214   
Video_1                          0.236111  0.0  1.0    0.0  0.427672   
WIN_20210323_19_17_40_Pro        0.472222  0.0  1.0    0.0  0.502731   
WIN_20210329_10_16_02_Pro        0.930556  0.0  1.0    1.0  0.255992   
WIN_20210330_13_10_29_Pro        0.958904  0.0  1.0    1.0  0.199886   
WIN_20210331_21_22_52_Pro        0.901408  0.0  1.0    1.0  0.300235   
WIN_20210402_14_27_50_Pro        1.847222  1.0  2.0    2.0  0.362298   
WIN_20210402_19_04_53_Pro        0.680556  0.0  1.0    1.0  0.469533   
WIN_20210403_18_49_15_Pro        0.722222  0.0  1.0    1.0  0.451046   
WIN_20210404_10_58_27_Pro        0.698630  0.0  1.0    1.0  0.462028   
WIN_20210404_21_41_12_Pro        0.712329  0.0  1.0    1.0  0.455810   
WIN_20210405_15_09_16_Pro        0.219178  0.0  1.0    0.0  0.416552   
WIN_20210406_15_06_15_Pro        0.152778  0.0  1.0    0.0  0.362298   
WIN_20210406_18_35_52_Pro        0.194444  0.0  1.0    0.0  0.398550   
WIN_20210406_18_49_10_Pro        0.166667  0.0  1.0    0.0  0.375293   
WIN_20210406_21_05_52_Pro        0.225352  0.0  1.0    0.0  0.420788   
WIN_20210407_09_04_05_Pro        0.780822  0.0  1.0    1.0  0.416552   
WIN_20210407_14_54_56_Pro_edit2  0.472222  0.0  1.0    0.0  0.502731   
WIN_20210408_11_48_58_Pro        0.958333  0.0  1.0    1.0  0.201229   
WIN_20210408_14_00_44_Pro        0.450704  0.0  1.0    0.0  0.501105   
WIN_20210408_14_02_19_Pro        0.808219  0.0  2.0    1.0  0.827606   
WIN_20210408_14_11_32_Pro        1.000000  1.0  1.0    1.0  0.000000   
WIN_20210408_15_20_51_Pro        0.575342  0.0  2.0    1.0  0.575370   
WIN_20210408_16_04_32_Pro        0.722222  0.0  1.0    1.0  0.451046   
WIN_20210409_10_26_11_Pro        0.222222  0.0  1.0    0.0  0.418657   
WIN_20210413_15_38_01_Pro        0.458333  0.0  1.0    0.0  0.501757   
WIN_20210414_06_24_52_Pro        0.027778  0.0  1.0    0.0  0.165489   
WIN_20210415_15_41_24_Pro        0.150685  0.0  1.0    0.0  0.360217   
WIN_20210416_08_06_54_Pro        0.684932  0.0  1.0    1.0  0.467758   
WIN_20210417_14_53_12_Pro        0.191781  0.0  1.0    0.0  0.396426   

                                                                              
                                percentil25 percentil75   kurtosis      skew  
video_name                                                                    
Test_pour_AFPA                          1.0         1.0  12.809701 -3.848337  
Video_1                                 0.0         0.0  -0.455615  1.242733  
WIN_20210323_19_17_40_Pro               0.0         1.0  -1.987616  0.111283  
WIN_20210329_10_16_02_Pro               1.0         1.0   9.474627 -3.387422  
WIN_20210330_13_10_29_Pro               1.0         1.0  19.376190 -4.623439  
WIN_20210331_21_22_52_Pro               1.0         1.0   5.252232 -2.692997  
WIN_20210402_14_27_50_Pro               2.0         2.0   1.725782 -1.930229  
WIN_20210402_19_04_53_Pro               0.0         1.0  -1.400177 -0.774482  
WIN_20210403_18_49_15_Pro               0.0         1.0  -1.015385 -0.992278  
WIN_20210404_10_58_27_Pro               0.0         1.0  -1.250446 -0.865768  
WIN_20210404_21_41_12_Pro               0.0         1.0  -1.119963 -0.938103  
WIN_20210405_15_09_16_Pro               0.0         0.0  -0.156798  1.357646  
WIN_20210406_15_06_15_Pro               0.0         0.0   1.725782  1.930229  
WIN_20210406_18_35_52_Pro               0.0         0.0   0.384236  1.544097  
WIN_20210406_18_49_10_Pro               0.0         0.0   1.200000  1.788854  
WIN_20210406_21_05_52_Pro               0.0         0.0  -0.271591  1.314690  
WIN_20210407_09_04_05_Pro               1.0         1.0  -0.156798 -1.357646  
WIN_20210407_14_54_56_Pro_edit2         0.0         1.0  -1.987616  0.111283

In [120]:
Xy = df_ypredict.merge(df_annotations_stress, on='video_name')
X = Xy.iloc[:,:-1].set_index('video_name')
y = Xy.iloc[:,-1]

In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()

models_list = [
                LogisticRegression(multi_class='multinomial', fit_intercept=True, random_state=42),
                Pipeline(steps=[('pca', pca), ('logistic', LogisticRegression(multi_class='multinomial', fit_intercept=True))]),
                KNeighborsClassifier(),
                Pipeline(steps=[('pca', pca), ('knn', KNeighborsClassifier())]),
                RandomForestClassifier(random_state = 42, n_jobs=-1)
                ]

parameters_list = [
                    {'C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'class_weight' : [None, 'balanced']},
                    {'pca__n_components': [1, 2, 3, 4],
                        'logistic__C': [0.01, 0.05, 0.1, 0.5, 1, 2, 3, 4 , 5, 10], 'logistic__class_weight' : [None, 'balanced']},
                    {'n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'weights' : ['uniform', 'distance'], 'p': [1, 2]},
                    {'pca__n_components': [1, 2, 3, 4],
                        'knn__n_neighbors': [4, 5, 6, 7, 8, 9, 10, 11, 12,  15, 20], 'knn__weights' : ['uniform', 'distance'],                              'knn__p': [1, 2]},
                    {'n_estimators': [50, 100, 150, 200], 'max_depth':[3, 4, 5, 6, 10, 15, 20], 'class_weight':[None,'balanced']}
                    ]

groups = X.reset_index()['video_name']
loo = LeaveOneGroupOut()
cv_loo = loo.split(X, y, groups)

In [122]:
best_result, y_predict, result_list = runGridSearchClassifiers(X, y, cv_loo, models_list, parameters_list, 
                                                                output_predict=True, n_jobs=-1, verbose=True)

Fitting 30 folds for each of 20 candidates, totalling 600 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 536 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    6.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Best estimator LogisticRegression(C=0.05, class_weight='balanced', multi_class='multinomial',
                   random_state=42)
Best results 0.4666666666666667
Best params {'C': 0.05, 'class_weight': 'balanced'}
accuracy (mean, std) 0.4666666666666667 0.49888765156985887
f1 (mean, std) 0.4666666666666667 0.49888765156985887
balanced accuracy (mean, std) 0.4666666666666667 0.49888765156985887
precision (mean, std) 0.4666666666666667 0.49888765156985887
recall (mean, std) 0.4666666666666667 0.49888765156985887

Fitting 30 folds for each of 80 candidates, totalling 2400 fits
[Parallel

In [123]:
best_result

{'best_estimator': LogisticRegression(C=0.05, class_weight='balanced', multi_class='multinomial',
                    random_state=42),
 'best_score': 0.4666666666666667,
 'best_params': {'C': 0.05, 'class_weight': 'balanced'},
 'mean_test_f1_score': 0.4666666666666667,
 'std_test_f1_score': 0.49888765156985887,
 'mean_test_accuracy_score': 0.4666666666666667,
 'std_test_accuracy_score': 0.49888765156985887,
 'mean_test_balanced_accuracy_score': 0.4666666666666667,
 'std_test_balanced_accuracy_score': 0.49888765156985887,
 'mean_test_precision': 0.4666666666666667,
 'std_test_precision': 0.49888765156985887,
 'mean_test_recall': 0.4666666666666667,
 'std_test_recall': 0.49888765156985887}